In [1]:
#!/usr/bin/env python3
# ============================================================
# 🧩 merge_datasets_for_tokens.py
# Une dataset_prototipo.csv + eval_set_inclusion_v2.csv
# para análisis conjunto de tokens sospechosos
# ============================================================

import pandas as pd
from urllib.parse import urlparse

# ------------------------------------------------------------
# Rutas de entrada/salida
# ------------------------------------------------------------
PROTO_PATH = "dataset/dataset_prototipo.csv"
INCL_PATH = "dataset/eval_set_inclusion_v2.csv"
OUTPUT_PATH = "data/processed/dataset_tokens_full.csv"

# ------------------------------------------------------------
# Carga y normalización
# ------------------------------------------------------------
print("📥 Cargando datasets...")
proto = pd.read_csv(PROTO_PATH)
incl = pd.read_csv(INCL_PATH)

# Normalizar estructura mínima
common_cols = [c for c in proto.columns if c in incl.columns]
proto = proto[common_cols]
incl = incl[common_cols]

df = pd.concat([proto, incl], ignore_index=True)
print(f"Total inicial: {len(df)} URLs")

# ------------------------------------------------------------
# Limpieza básica
# ------------------------------------------------------------
df = df.drop_duplicates(subset=["url"])
df["url"] = df["url"].astype(str).str.strip()

# Extraer dominio base para deduplicación adicional
df["domain_base"] = df["url"].apply(lambda x: urlparse(x).netloc.replace("www.", ""))
df = df.drop_duplicates(subset=["domain_base", "label"], keep="first")

print(f"Tras deduplicación: {len(df)} URLs únicas por dominio y clase")

# ------------------------------------------------------------
# Validación de etiquetas
# ------------------------------------------------------------
if "label" not in df.columns:
    raise ValueError("❌ Falta columna 'label' en los datasets.")

assert df["label"].isin([0,1]).all(), "❌ Hay valores de label no válidos."

# ------------------------------------------------------------
# Exportación
# ------------------------------------------------------------
df.to_csv(OUTPUT_PATH, index=False)
print(f"✅ Dataset combinado guardado en: {OUTPUT_PATH}")
print(df["label"].value_counts().rename({0: "legítimas", 1: "phishing"}))



📥 Cargando datasets...
Total inicial: 500 URLs
Tras deduplicación: 362 URLs únicas por dominio y clase
✅ Dataset combinado guardado en: data/processed/dataset_tokens_full.csv
label
phishing     240
legítimas    122
Name: count, dtype: int64


In [3]:
# ============================================================
# 🧠 review_tokens.ipynb
# Revisión y validación manual de tokens candidatos
# ============================================================

import pandas as pd
import numpy as np
from urllib.parse import urlparse
from IPython.display import display, Markdown

# ------------------------------------------------------------
# 📥 Carga de datos
# ------------------------------------------------------------
TOKENS_PATH = "data/processed/tokens_candidates.csv"
DATA_PATH = "data/processed/dataset_tokens_full.csv"

df_tokens = pd.read_csv(TOKENS_PATH)
df_urls = pd.read_csv(DATA_PATH)

print(f"Tokens cargados: {len(df_tokens)}")
print(f"URLs disponibles: {len(df_urls)}")

# ------------------------------------------------------------
# 🔍 Funciones auxiliares
# ------------------------------------------------------------

def show_examples(token, n=5):
    """Muestra ejemplos de URLs (phish y legit) donde aparece el token."""
    subset = df_urls[df_urls["url"].str.contains(token, case=False, na=False)]
    if subset.empty:
        display(Markdown(f"❌ **No se encontraron ejemplos para:** `{token}`"))
        return
    
    phish = subset[subset["label"] == 1].head(n)
    legit = subset[subset["label"] == 0].head(n)

    display(Markdown(f"### 🧩 Token: `{token}`"))
    display(Markdown(f"**Total URLs encontradas:** {len(subset)}"))
    
    if not phish.empty:
        display(Markdown("#### 🔴 Ejemplos Phishing"))
        display(phish[["url"]])
    if not legit.empty:
        display(Markdown("#### 🟢 Ejemplos Legítimos"))
        display(legit[["url"]])
    
    print("-" * 60)

def filter_tokens(min_freq=2, min_logodds=0.3, hint="phish"):
    """Filtra tokens según frecuencia y orientación (phish/legit)."""
    subset = df_tokens.copy()
    subset = subset[(subset["total"] >= min_freq) & (subset["class_hint"] == hint)]
    subset = subset[subset["log_odds"].abs() >= min_logodds]
    subset = subset.sort_values("log_odds", ascending=False)
    return subset

# ------------------------------------------------------------
# 📊 Vista general y exploración
# ------------------------------------------------------------

display(Markdown("## 📊 Resumen general de tokens"))
display(df_tokens.describe())

display(Markdown("### Top 20 tokens más asociados a phishing"))
top_phish = filter_tokens(min_freq=2, min_logodds=0.3, hint="phish").head(20)
display(top_phish[["token","log_odds","phish_freq","legit_freq","is_seed"]])

display(Markdown("### Top 20 tokens más asociados a legítimos"))
top_legit = filter_tokens(min_freq=2, min_logodds=0.3, hint="legit").head(20)
display(top_legit[["token","log_odds","phish_freq","legit_freq","is_seed"]])

# ------------------------------------------------------------
# 🧩 Ejemplo de revisión manual (ajusta el token)
# ------------------------------------------------------------

# 👉 Cambia este valor para inspeccionar cualquier token
token_to_review = "verificar"
show_examples(token_to_review, n=5)

# ------------------------------------------------------------
# 🧾 Revisión masiva interactiva
# ------------------------------------------------------------

# Creamos un dataframe para revisión y etiquetas manuales
df_review = df_tokens.copy()
df_review["approved"] = 0
df_review["notes"] = ""

# Criterio de filtrado inicial (puedes cambiarlo)
df_review = df_review[
    (df_review["class_hint"] == "phish") &
    (df_review["log_odds"] > 0.3) &
    (df_review["total"] >= 2)
].reset_index(drop=True)

print(f"Tokens candidatos a revisar: {len(df_review)}")

# Ejemplo de cómo marcar manualmente un token aprobado:
# df_review.loc[df_review["token"] == "verificar", "approved"] = 1
# df_review.loc[df_review["token"] == "verificar", "notes"] = "Alta frecuencia en phishing Correos"

# ------------------------------------------------------------
# 💾 Guardar revisión (ejecutar cuando termines la tanda)
# ------------------------------------------------------------

OUTPUT_REVIEW = "data/interim/tokens_reviewed.csv"
df_review.to_csv(OUTPUT_REVIEW, index=False)
print(f"✅ Archivo de revisión guardado en: {OUTPUT_REVIEW}")


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/tokens_candidates.csv'